<a href="https://colab.research.google.com/github/IshaanKetchup/ASLtoEnglish/blob/main/CNN%20Deepfake%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN BASED DEEPFAKE DETECTION

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/deepfake-and-real-images


##Loading Data

In [8]:
import tensorflow as tf
import os

# Set dataset path
dataset_path = path  # replace with actual path if needed
img_size = (128, 128)
batch_size = 32

# Create datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(dataset_path, "Dataset/Train"),
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(dataset_path, "Dataset/Validation"),
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary"
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(dataset_path, "Dataset/Test"),
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary"
)

# Optional performance boost
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


##CNN Architecture

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=img_size + (3,)), #image normalization (0-255 -> 0-1) and shape definition (128x128, 3 channels RGB)
    tf.keras.layers.Conv2D(32, 3, activation='relu'), # Input:(128, 128, 3), Output:(126, 126, 32) 126 = 128 - 3 + 1
    tf.keras.layers.MaxPooling2D(), # Input:(126, 126, 32), Output:(63, 63, 32)

    tf.keras.layers.Conv2D(64, 3, activation='relu'), # Input:(63, 63, 32), Output:(61, 61, 64)
    tf.keras.layers.MaxPooling2D(), # Input:(61, 61, 64), Output:(30, 30, 64)

    tf.keras.layers.Conv2D(128, 3, activation='relu'), # Input:(30, 30, 64), Output:(28, 28, 128)
    tf.keras.layers.MaxPooling2D(), # Input:(28, 28, 128), Output:(14, 14, 128)

    tf.keras.layers.Flatten(), # 3D Tensor to 2D Vector-- Input:(14, 14, 128), Output:14×14×128 = 25088
    tf.keras.layers.Dense(128, activation='relu'), # Fully connected layer -- Input: 25088, Output: 128
    tf.keras.layers.Dropout(0.5), # Randomly set 50% of inputs to 0 during training to prevent overfitting
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary Classifier
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


##Compiling the Model

In [10]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


##Training the Model

In [11]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 519s 117ms/step - accuracy: 0.8034 - loss: 0.4071 - val_accuracy: 0.8640 - val_loss: 0.2971
Epoch 2/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 207s 37ms/step - accuracy: 0.9306 - loss: 0.1727 - val_accuracy: 0.9117 - val_loss: 0.2063
Epoch 3/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 150s 34ms/step - accuracy: 0.9457 - loss: 0.1336 - val_accuracy: 0.9230 - val_loss: 0.1809
Epoch 4/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 160s 37ms/step - accuracy: 0.9548 - loss: 0.1140 - val_accuracy: 0.9201 - val_loss: 0.2059
Epoch 5/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 158s 36ms/step - accuracy: 0.9606 - loss: 0.0986 - val_accuracy: 0.9091 - val_loss: 0.2409


##Evaluation

In [12]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)


341/341 ━━━━━━━━━━━━━━━━━━━━ 28s 83ms/step - accuracy: 0.8781 - loss: 0.3196
Test accuracy: 0.8759284615516663


##Saving the Model

In [14]:
model.save("deepfake_cnn_model.keras")  # Creates a folder with all files
